Lectura de los datos del escenario de test

In [16]:
import pandas as pd

pd.set_option('display.max_columns', None)
df = pd.read_parquet("scenario_001.parquet")
print(df.columns)

Index(['df', 'squawk', 'bds60_bds', 'bds60_heading', 'bds60_IAS', 'bds60_Mach',
       'bds60_vrate_barometric', 'bds60_vrate_inertial', 'icao24', 'ts',
       ...
       'bds21_registration', 'bds40_target_source', 'bds17_bds45',
       'bds17_bds54', 'bds17_bds55', 'bds17_bds56', 'year', 'month', 'day',
       'hour'],
      dtype='object', length=134)


Los siguientes pares de columnas se refieren a lo mismo pero se diferencian en la fuente de la que provienen (más adelante se combinarán en una sola)

In [7]:
# No hay ninguna fila en la que ambas sean nulas
df[["callsign", "bds20_callsign"]][df[["callsign", "bds20_callsign"]].notnull().any(axis=1)]

,callsign,bds20_callsign
18,EDW288G,None
24,TVF54DQ,None
58,None,AEA7701
59,DLH36N,None
64,ANE86ZR,None
...,...,...
47946,None,IBE0267
47947,None,ANE30KP
47955,None,IBS1731
47986,None,IBS1731


In [8]:
# No hay ninguna fila en la que ambas sean nulas
df[["groundspeed", "bds50_groundspeed"]][df[["groundspeed", "bds50_groundspeed"]].notnull().any(axis=1)]

,groundspeed,bds50_groundspeed
11,5.000000,NaN
13,249.419326,NaN
16,1.750000,NaN
17,NaN,400.0
23,7.750000,NaN
...,...,...
48012,298.529731,NaN
48017,477.582454,NaN
48020,NaN,466.0
48025,NaN,414.0


In [9]:
# Tienen el mismo valor cuando las dos son no nulas
df[["altitude", "bds05_altitude"]][df[["altitude", "bds05_altitude"]].notnull().all(axis=1)]

,altitude,bds05_altitude
147,39025.0,39025.0
192,39000.0,39000.0
193,15325.0,15325.0
279,9025.0,9025.0
300,31000.0,31000.0
...,...,...
47746,31000.0,31000.0
47873,38025.0,38025.0
47890,39000.0,39000.0
47896,21950.0,21950.0


In [10]:
df["tc"].unique() # typecode?

array([nan, 11.,  7.,  8., 12.,  6., 18.,  5.])

In [11]:
list(df.columns)

['df',
 'squawk',
 'bds60_bds',
 'bds60_heading',
 'bds60_IAS',
 'bds60_Mach',
 'bds60_vrate_barometric',
 'bds60_vrate_inertial',
 'icao24',
 'ts',
 'bds',
 'NACv',
 'groundspeed',
 'track',
 'vrate_src',
 'vertical_rate',
 'geo_minus_baro',
 'tc',
 'NUCp',
 'NICb',
 'altitude',
 'source',
 'parity',
 'lat_cpr',
 'lon_cpr',
 'lat_deg',
 'lon_deg',
 'bds40_bds',
 'bds40_selected_mcp',
 'bds40_barometric_setting',
 'bds40_selected_fms',
 'selected_altitude',
 'barometric_setting',
 'NACp',
 'tcas_operational',
 'bds50_bds',
 'bds50_roll',
 'bds50_track',
 'bds50_groundspeed',
 'bds50_track_rate',
 'bds50_TAS',
 'version',
 'NICa',
 'GVA',
 'SIL',
 'BAI',
 'HRD',
 'SILs',
 'bds45_bds',
 'bds45_turbulence',
 'bds45_wind_shear',
 'bds45_microburst',
 'bds45_icing',
 'bds45_wake_vortex',
 'bds45_static_temperature',
 'bds45_static_pressure',
 'bds45_radio_height',
 'tisb',
 'id',
 'wake_vortex',
 'callsign',
 'TAH',
 'selected_heading',
 'bds20_bds',
 'bds20_callsign',
 'subtype',
 'emergen

### Transformamos el escenario test para poder aplicarle el pipeline de preprocesamiento

Eliminación de columnas innecesarias

In [12]:
df_new = df[["ts", "icao24", "df", "tc", "callsign", "bds20_callsign", "lat_deg", "lon_deg", "altitude", "bds05_altitude", "groundspeed", "bds50_groundspeed"]]
df_new.head()

,ts,icao24,df,tc,callsign,bds20_callsign,lat_deg,lon_deg,altitude,bds05_altitude,groundspeed,bds50_groundspeed
0,1741503722828,3c618b,20,NaN,None,None,NaN,NaN,39000.0,NaN,NaN,NaN
1,1741503722828,347256,20,NaN,None,None,NaN,NaN,33000.0,NaN,NaN,NaN
2,1741503722828,3451d8,17,11.0,None,None,40.766235,-3.148132,9100.0,NaN,NaN,NaN
3,1741503722828,4952c6,20,NaN,None,None,NaN,NaN,39025.0,NaN,NaN,NaN
4,1741503722828,347256,21,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN


Renombrar y combinar columnas que ya existen en el escenario test

In [13]:
# Cambiamos el formato de al columna timestamp
df_new["ts"] = pd.to_datetime(df_new["ts"], unit='ms') # para que se pueda leer también con spark

# Pasamos el typecode y downlink format a enteros
df_new["tc"] = df_new["tc"].astype("Int64")
df_new["df"] = df_new["df"].astype("Int64")

# Combinamos las dos columnas de callsign en una sola
df_new["Callsign"] = df_new["callsign"].fillna(df_new["bds20_callsign"])
df_new = df_new.drop(columns=["callsign", "bds20_callsign"])

# Combinamos las dos columnas de speed en una sola
df_new["Speed"] = df_new["groundspeed"].fillna(df_new["bds50_groundspeed"])
df_new = df_new.drop(columns=["groundspeed", "bds50_groundspeed"])

# Combinamos las dos columnas de altitude en una sola
df_new["Altitude (ft)"] = df_new["altitude"].fillna(df_new["bds05_altitude"])
df_new = df_new.drop(columns=["altitude", "bds05_altitude"])

df_new = df_new.rename(columns={"ts": "Timestamp (date)", "icao24": "ICAO", "df": "Downlink Format", "tc": "Typecode",
                                 "lat_deg": "lat", "lon_deg": "lon"})

df_new.head()

/tmp/ipykernel_2780/3058425714.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["ts"] = pd.to_datetime(df_new["ts"], unit='ms') # para que se pueda leer también con spark
/tmp/ipykernel_2780/3058425714.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["tc"] = df_new["tc"].astype("Int64")
/tmp/ipykernel_2780/3058425714.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,Timestamp (date),ICAO,Downlink Format,Typecode,lat,lon,Callsign,Speed,Altitude (ft)
0,2025-03-09 07:02:02.828,3c618b,20,<NA>,NaN,NaN,None,NaN,39000.0
1,2025-03-09 07:02:02.828,347256,20,<NA>,NaN,NaN,None,NaN,33000.0
2,2025-03-09 07:02:02.828,3451d8,17,11,40.766235,-3.148132,None,NaN,9100.0
3,2025-03-09 07:02:02.828,4952c6,20,<NA>,NaN,NaN,None,NaN,39025.0
4,2025-03-09 07:02:02.828,347256,21,<NA>,NaN,NaN,None,NaN,NaN


In [14]:
df_new.dtypes

Timestamp (date)    datetime64[ms]
ICAO                        object
Downlink Format              Int64
Typecode                     Int64
lat                        float64
lon                        float64
Callsign                    object
Speed                      float64
Altitude (ft)              float64
dtype: object

Sacar las variables que faltan:
- _Flight status_
- _TurbulenceCategory_

In [33]:
import numpy as np

# Flight Status basado en Downlink Format y Typecode
conditions_fs = [
    # ADS-B messages (DF 17 or 18) - Use Typecode range logic from processADS_B
    (df_new["Downlink Format"].isin([17, 18])) & (df_new["Typecode"].isin(range(1, 5))), # TC 1-4 (Tierra)
    (df_new["Downlink Format"].isin([17, 18])) & (df_new["Typecode"].isin(range(5, 23))), # TC 5-22 (Airborne) 

    df_new["Downlink Format"].isin([4, 20]), # Suponemos en tierra?

    df_new["Downlink Format"].isin([5, 21]) # Airbone?
]
results_fs = [
    'on-ground',
    'airborne',
    'on-ground',
    'airborne'
]

df_new["Flight Status"] = np.select(conditions_fs, results_fs, default='NaN') 


In [34]:

# Turbulence Category basado en bds45_turbulence (Turbulence level 0-3)
# NOTA: Esto se basa en el dato de turbulencia reportada de BDS 4,5, NO en la categoría de aeronave/estela
# que tu decoder llama 'TurbulenceCategory'. Usamos bds45_turbulence del DataFrame original 'df'.
conditions_tc = [
    df["bds45_turbulence"].isna(), # Condición 1: Es NaN
    df["bds45_turbulence"] == 0,    # Condición 2: Nivel 0
    df["bds45_turbulence"] == 1,    # Condición 3: Nivel 1
    df["bds45_turbulence"] == 2,    # Condición 4: Nivel 2
    df["bds45_turbulence"] == 3     # Condición 5: Nivel 3
]
results_tc = [
    'Not Reported',           # Resultado si Condición 1 es True
    'None Reported',          # Resultado si Condición 2 es True
    'Light Turbulence',       # Resultado si Condición 3 es True
    'Moderate Turbulence',    # Resultado si Condición 4 es True
    'Severe Turbulence'       # Resultado si Condición 5 es True
]

df_new["TurbulenceCategory"] = np.select(conditions_tc, results_tc, default='Unknown Level') # Default for unexpected values


In [36]:
df_new

,Timestamp (date),ICAO,Downlink Format,Typecode,lat,lon,Callsign,Speed,Altitude (ft),Flight Status,TurbulenceCategory
0,2025-03-09 07:02:02.828,3c618b,20,<NA>,NaN,NaN,None,NaN,39000.0,on-ground,Not Reported
1,2025-03-09 07:02:02.828,347256,20,<NA>,NaN,NaN,None,NaN,33000.0,on-ground,Unknown Level
2,2025-03-09 07:02:02.828,3451d8,17,11,40.766235,-3.148132,None,NaN,9100.0,airborne,Not Reported
3,2025-03-09 07:02:02.828,4952c6,20,<NA>,NaN,NaN,None,NaN,39025.0,on-ground,Not Reported
4,2025-03-09 07:02:02.828,347256,21,<NA>,NaN,NaN,None,NaN,NaN,airborne,Not Reported
...,...,...,...,...,...,...,...,...,...,...,...
48023,2025-03-09 07:04:02.694,39d301,20,<NA>,NaN,NaN,None,NaN,39000.0,on-ground,Not Reported
48024,2025-03-09 07:04:02.694,34620c,21,<NA>,NaN,NaN,None,NaN,NaN,airborne,Not Reported
48025,2025-03-09 07:04:02.694,4b1a2a,21,<NA>,NaN,NaN,None,414.000000,NaN,airborne,Not Reported
48026,2025-03-09 07:04:02.694,34134e,17,<NA>,NaN,NaN,None,435.765992,NaN,NaN,Not Reported
